<a href="https://colab.research.google.com/github/HerlanAssis/P-median-problem/blob/main/P_Median_Problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problema das *p-medianas*

"A localização de p-medianas é um problema clássico de otimização combinatória. O objetivo é localizar em uma rede p nós (denominados medianas), de forma a minimizar a soma das distâncias de cada nó de demanda até sua mediana mais próxima" (Senne e Lorena, 2003). 

O problema de p-medianas pode ser modelado como o seguinte problema:

![](https://github.com/HerlanAssis/P-median-problem/blob/main/pmediana.png?raw=true)

### Instalando dependências

Para executarmos um exemplo do problema das *p-medianas* precisamos das seguintes dependências:
* ibmdecisionoptimization
* cplex
* docplex

Para fazer isso vamos utilizar o ambiente virtual *anaconda*, como ele conseguiremos configurar um abiente utilizando um notabook python.

*OBS: Para configurar o ambiente com o conda utilizei o seguinte link [Stackoverflow: install conda package to google colab](https://stackoverflow.com/questions/59330876/install-conda-package-to-google-colab)*

In [1]:
'''
There are 2 problems that must be solved:

ujson will normally upgrade to python 3.7, must avoid this.
path to conda library is changed, must update it.
For 1, you need to add python=3.6 to conda install.

For 2, you need to add path to /usr/local/lib/python3.6/site-packages

Here's the new code
'''

# same
!wget -c https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
!chmod +x Miniconda3-4.5.4-Linux-x86_64.sh
!bash ./Miniconda3-4.5.4-Linux-x86_64.sh -b -f -p /usr/local
# update 1
!conda install -q -y --prefix /usr/local python=3.6 ujson
# update 2
import sys
sys.path.append('/usr/local/lib/python3.6/site-packages')

--2020-11-10 18:23:00--  https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58468498 (56M) [application/x-sh]
Saving to: ‘Miniconda3-4.5.4-Linux-x86_64.sh’

Miniconda3-4.5.4-Li 100%[===================>]  55.76M   140MB/s    in 0.4s    

2020-11-10 18:23:01 (140 MB/s) - ‘Miniconda3-4.5.4-Linux-x86_64.sh’ saved [58468498/58468498]

PREFIX=/usr/local
installing: python-3.6.5-hc3d631a_2 ...
Python 3.6.5 :: Anaconda, Inc.
installing: ca-certificates-2018.03.07-0 ...
installing: conda-env-2.6.0-h36134e3_1 ...
installing: libgcc-ng-7.2.0-hdf63c60_3 ...
installing: libstdcxx-ng-7.2.0-hdf63c60_3 ...
installing: libffi-3.2.1-hd88cf55_4 ...
installing: ncurses-6.1-hf484d3e_0 ...
installing: openssl-1.0.2o-h20670df_0 ...
installing: tk-8.6

In [2]:
!conda install -c ibmdecisionoptimization cplex docplex --yes
from docplex.cp.model import CpoModel

Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.4
  latest version: 4.9.1

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /usr/local

  added / updated specs: 
    - cplex
    - docplex


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    cryptography-3.1.1         |   py36h1ba5d50_0         612 KB
    requests-2.24.0            |             py_0          54 KB
    urllib3-1.25.11            |             py_0          93 KB
    cffi-1.14.3                |   py36he30daa8_0         224 KB
    docplex-2.16.196           |           py36_0        1011 KB  ibmdecisionoptimization
    cplex-12.10     

### Instâncias de exemplo

In [22]:
'''
p: número de facilidades a serem escolhidas
m: clientes
n: numeros de medianas possíveis
d: matriz de distâncias do cliente para facilidade

----- exemplo de instância
m = 5
n = 8
p = 3
d = [[25.0, 21.67, 21.67, 13.33, 8.33, 18.33, 8.33, 21.67],
     [6.67, 16.67, 11.67, 20.0, 15.0, 15.0, 8.33, 15.0],
     [6.67, 16.67, 15.0, 13.33, 13.33, 16.67, 21.67, 18.33],
     [25.0, 8.33, 11.67, 23.33, 16.67, 21.67, 20.0, 18.33],
     [18.33, 21.67, 13.33, 6.67, 21.67, 23.33, 18.33, 8.33]]
'''

d = [[0, 400.00, 1300.00, 1700.00, 282.84, 1513.27, 400.00, 565.69, 1360.15, 1746.42, 2400.00, 2433.11, 2729.47, 2941.09, 2607.68, 3001.67, 2800.00, 2828.43, 3087.07, 3275.67],
[400.00, 0, 900.00, 1300.00, 282.84, 1118.03, 565.69, 400.00, 984.89, 1360.15, 2433.11, 2400.00, 2563.20, 2729.47, 2607.68, 2823.12, 2828.43, 2800.00, 2941.09, 3087.07],
[1300.00, 900.00, 0, 400.00, 1118.03, 282.84, 1360.15, 984.89, 400.00, 565.69, 2729.47, 2563.20, 2400.00, 2433.11, 2823.12, 2607.68, 3087.07, 2941.09, 2800.00, 2828.43],
[1700.00, 1300.00, 400.00, 0, 1513.27, 282.84, 1746.42, 1360.15, 565.69, 400.00, 2941.09, 2729.47, 2433.11, 2400.00, 3001.67, 2607.68, 3275.67, 3087.07, 2828.43, 2800.00],
[282.84, 282.84, 1118.03, 1513.27, 0, 1300.00, 282.84, 282.84, 1118.03, 1513.27, 2209.07, 2209.07, 2459.67, 2662.71, 2400.00, 2729.47, 2607.68, 2607.68, 2823.12, 3001.67],
[1513.27, 1118.03, 282.84, 282.84, 1300.00, 0, 1513.27, 1118.03, 282.84, 282.84, 2662.71, 2459.67, 2209.07, 2209.07, 2729.47, 2400.00, 3001.67, 2823.12, 2607.68, 2607.68],
[400.00, 565.69, 1360.15, 1746.42, 282.84, 1513.27, 0, 400.00, 1300.00, 1700.00, 2000.00, 2039.61, 2385.37, 2624.88, 2209.07, 2662.71, 2400.00, 2433.11, 2729.47, 2941.09],
[565.69, 400.00, 984.89, 1360.15, 282.84, 1118.03, 400.00, 0, 900.00, 1300.00, 2039.61, 2000.00, 2193.17, 2385.37, 2209.07, 2459.67, 2433.11, 2400.00, 2563.20, 2729.47],
[1360.15, 984.89, 400.00, 565.69, 1118.03, 282.84, 1300.00, 900.00, 0, 400.00, 2385.37, 2193.17, 2000.00, 2039.61, 2459.67, 2209.07, 2729.47, 2563.20, 2400.00, 2433.11],
[1746.42, 1360.15, 565.69, 400.00, 1513.27, 282.84, 1700.00, 1300.00, 400.00, 0, 2624.88, 2385.37, 2039.61, 2000.00, 2662.71, 2209.07, 2941.09, 2729.47, 2433.11, 2400.00],
[2400.00, 2433.11, 2729.47, 2941.09, 2209.07, 2662.71, 2000.00, 2039.61, 2385.37, 2624.88, 0, 400.00, 1300.00, 1700.00, 282.84, 1513.27, 400.00, 565.69, 1360.15, 1746.42],
[2433.11, 2400.00, 2563.20, 2729.47, 2209.07, 2459.67, 2039.61, 2000.00, 2193.17, 2385.37, 400.00, 0, 900.00, 1300.00, 282.84, 1118.03, 565.69, 400.00, 984.89, 1360.15],
[2729.47, 2563.20, 2400.00, 2433.11, 2459.67, 2209.07, 2385.37, 2193.17, 2000.00, 2039.61, 1300.00, 900.00, 0, 400.00, 1118.03, 282.84, 1360.15, 984.89, 400.00, 565.69],
[2941.09, 2729.47, 2433.11, 2400.00, 2662.71, 2209.07, 2624.88, 2385.37, 2039.61, 2000.00, 1700.00, 1300.00, 400.00, 0, 1513.27, 282.84, 1746.42, 1360.15, 565.69, 400.00],
[2607.68, 2607.68, 2823.12, 3001.67, 2400.00, 2729.47, 2209.07, 2209.07, 2459.67, 2662.71, 282.84, 282.84, 1118.03, 1513.27, 0, 1300.00, 282.84, 282.84, 1118.03, 1513.27],
[3001.67, 2823.12, 2607.68, 2607.68, 2729.47, 2400.00, 2662.71, 2459.67, 2209.07, 2209.07, 1513.27, 1118.03, 282.84, 282.84, 1300.00, 0, 1513.27, 1118.03, 282.84, 282.84],
[2800.00, 2828.43, 3087.07, 3275.67, 2607.68, 3001.67, 2400.00, 2433.11, 2729.47, 2941.09, 400.00, 565.69, 1360.15, 1746.42, 282.84, 1513.27, 0, 400.00, 1300.00, 1700.00],
[2828.43, 2800.00, 2941.09, 3087.07, 2607.68, 2823.12, 2433.11, 2400.00, 2563.20, 2729.47, 565.69, 400.00, 984.89, 1360.15, 282.84, 1118.03, 400.00, 0, 900.00, 1300.00],
[3087.07, 2941.09, 2800.00, 2828.43, 2823.12, 2607.68, 2729.47, 2563.20, 2400.00, 2433.11, 1360.15, 984.89, 400.00, 565.69, 1118.03, 282.84, 1300.00, 900.00, 0, 400.00],
[3275.67, 3087.07, 2828.43, 2800.00, 3001.67, 2607.68, 2941.09, 2729.47, 2433.11, 2400.00, 1746.42, 1360.15, 565.69, 400.00, 1513.27, 282.84, 1700.00, 1300.00, 400.00, 0]]

m = len(d)
n = len(d)
p = 5

### Código Principal

In [23]:
clients = range(m)
places = range(n)

mdl = CpoModel()

x = [[mdl.integer_var(min=0, max=1, name="X[{}][{}] {}".format(place, client, d[client][place])) for place in places] for client in clients]

y = [mdl.integer_var(min=0, max=1, name="Y[{}] {}".format(place, place)) for place in places]

# Função objetivo
fo = mdl.sum(d[i][j]*x[i][j] for i in clients for j in places)
mdl.add(mdl.minimize(fo))

# sujeito a
mdl.add(mdl.sum([x[i][j] for j in places]) == 1 for i in clients)
mdl.add(x[i][j]-y[j] <= 0 for i in clients for j in places)
mdl.add(mdl.sum(y[j] for j in places) == p)

print("\nImprimindo solução....")
msol = mdl.solve(TimeLimit=60, Workers=1)

if msol:
    print(msol.print_solution())
    print("Status: " + msol.get_solve_status())
else:
    print("Nenhuma solução encontrada")


Imprimindo solução....
-------------------------------------------------------------------------------
Model constraints: 421, variables: integer: 420, interval: 0, sequence: 0
Solve status: Feasible
Search status: SearchStopped, stop cause: SearchStoppedByLimit
Solve time: 60.05 sec
-------------------------------------------------------------------------------
Objective values: (4242.6,), bounds: (282.825,), gaps: (0.933337,)
X[0][0] 0=0
X[0][1] 400.0=0
X[0][2] 1300.0=0
X[0][3] 1700.0=0
X[0][4] 282.84=0
X[0][5] 1513.27=0
X[0][6] 400.0=0
X[0][7] 565.69=0
X[0][8] 1360.15=0
X[0][9] 1746.42=0
X[0][10] 2400.0=0
X[0][11] 2433.11=0
X[0][12] 2729.47=0
X[0][13] 2941.09=0
X[0][14] 2607.68=0
X[0][15] 3001.67=0
X[0][16] 2800.0=0
X[0][17] 2828.43=0
X[0][18] 3087.07=0
X[0][19] 3275.67=0
X[1][0] 400.0=0
X[1][1] 0=0
X[1][2] 900.0=0
X[1][3] 1300.0=0
X[1][4] 282.84=0
X[1][5] 1118.03=0
X[1][6] 565.69=0
X[1][7] 400.0=0
X[1][8] 984.89=0
X[1][9] 1360.15=0
X[1][10] 2433.11=0
X[1][11] 2400.0=0
X[1][12] 256

### créditos
Este notebook foi feito com auxílio dos vídeos:

[1 - Network Optimization | P-Median Problem](https://www.youtube.com/watch?v=GytzJiuYUe4)

[Modelando o Problema de p medianas usando o CPLEX (aula)](https://www.youtube.com/watch?v=Fc9SsoPRDxo)


### Referências
* SENNE, Edson Luiz França; LORENA, Luiz Antonio Nogueira. Abordagens complementares para problemas de p-medianas. Production, v. 13, n. 3, p. 78-87, 2003.